In [1]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\Jerald\anaconda3\python.exe -m pip install --upgrade pip' command.


In [2]:
pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\Jerald\anaconda3\python.exe -m pip install --upgrade pip' command.


In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup as bs
import time
import re
from urllib.request import urlopen
import json
from selenium.webdriver.common.keys import Keys
import praw
import pandas as pd
import datetime as dt
from webdriver_manager.chrome import ChromeDriverManager

In [6]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [7]:
def scrape_url(source_path):
    page = 1
    url = []
    search_list = ['singapore','bivalent']
    while(page <= 2): 
        browser = webdriver.Chrome('chromedriver',options=chrome_options)
        browser.get(source_path)
        Pagelength = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        source = browser.page_source
        soup = bs(source, 'html.parser')
        body = soup.find('body')

        threads = soup.find_all("header", class_="search-result-header")
        next_page = soup.find_all("span", class_="nextprev")
        source_path = next_page[0].find("a").get('href')
                    
        for link in threads:
            title = link.find("a").text
            if re.compile('(?=.*' + ')(?=.*'.join(search_list) + ')', re.IGNORECASE).search(title): #re.IGNORECASE is used to ignore case
                url_string = link.find('a').get('href')
                url.append(url_string)
        page += 1            
    return url

In [8]:
source_path = "https://old.reddit.com/search?q=singapore+bivalent"
url_list = scrape_url(source_path)

SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 109
Current browser version is 111.0.5563.111 with binary path C:\Program Files\Google\Chrome\Application\chrome.exe
Stacktrace:
Backtrace:
	(No symbol) [0x00574AD3]
	(No symbol) [0x00509211]
	(No symbol) [0x003FD9CD]
	(No symbol) [0x004226D1]
	(No symbol) [0x0041D3F0]
	(No symbol) [0x0041A659]
	(No symbol) [0x00456677]
	(No symbol) [0x004562BC]
	(No symbol) [0x0044F936]
	(No symbol) [0x00424788]
	(No symbol) [0x00425C1D]
	GetHandleVerifier [0x007E9342+2502274]
	GetHandleVerifier [0x00817959+2692249]
	GetHandleVerifier [0x0081ABDC+2705180]
	GetHandleVerifier [0x00623480+643008]
	(No symbol) [0x0051208F]
	(No symbol) [0x00517AB8]
	(No symbol) [0x00517BA5]
	(No symbol) [0x0052273B]
	BaseThreadInitThunk [0x774400F9+25]
	RtlGetAppContainerNamedObjectPath [0x776D7BBE+286]
	RtlGetAppContainerNamedObjectPath [0x776D7B8E+238]


In [ ]:
url_list

In [ ]:
reddit = praw.Reddit(client_id='', \
                     client_secret='', \
                     user_agent='', \
                     username='', \
                     password='')

In [ ]:
topics_dict = { "author": [],
                "body": [],
                "score": [],
                "date": [],
                "source": []}

In [ ]:
def get_date(created):
    return dt.datetime.fromtimestamp(created)

In [ ]:
def process_topic_data(submission):
    comments = []
    submission.comments.replace_more(limit=None)
    for comment in submission.comments.list():
        timestamp = get_date(comment.created)
        topics_dict["author"].append(comment.author)
        topics_dict["body"].append(comment.body)
        topics_dict["score"].append(comment.score)
        topics_dict["date"].append(timestamp)
        topics_dict["source"].append("reddit")
    return comments

In [ ]:
for link in url_list:
    submission = reddit.submission(url=link)
    process_topic_data(submission)
#     collection_list.append(data)

In [ ]:
topics_data = pd.DataFrame(topics_dict)

In [ ]:
topics_data

In [ ]:
topics_data = topics_data.dropna()

In [ ]:
topics_data.to_json("reddit.json", orient='records', index=False)